In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [2]:
INPUT_DIR = "./ragtest/output"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [3]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 602


,level,title,type,description,source_id,degree,human_readable_id,id,size,graph_embedding,community,entity_type,top_level_node_id,x,y
0,0,IO_SAP,TECHNICAL_TERM,An IO control path interface that handles hard...,709bb69a81ecc1f51a9d4420b7de6772,1,0,c1c991eb4a97411eb3d036a31cac6f2f,1,None,None,None,c1c991eb4a97411eb3d036a31cac6f2f,0,0
1,0,INTER_SYSTEM_PROTOCOL,TECHNICAL_TERM,A protocol that enables various broadband powe...,709bb69a81ecc1f51a9d4420b7de6772,1,1,5d50bcf3d46d4778ae9c02df373727a7,1,None,None,None,5d50bcf3d46d4778ae9c02df373727a7,0,0
2,0,LOGICAL_NETWORK,TECHNICAL_TERM,LOGICAL_NETWORK is a network designed to enhan...,"709bb69a81ecc1f51a9d4420b7de6772,96fddbf4556ae...",4,2,91741f0c9864419ba6de55093a830cf0,4,None,1,None,91741f0c9864419ba6de55093a830cf0,0,0
3,0,LOW-LAYER_COMMUNICATION,TECHNICAL_TERM,Functions managed by the OSI layer 1 and layer...,709bb69a81ecc1f51a9d4420b7de6772,2,3,9b605578a3884c1dbe05bcf63f7044f5,2,None,None,None,9b605578a3884c1dbe05bcf63f7044f5,0,0
4,0,LOW-LAYER_COMMUNICATION_MODULE,TECHNICAL_TERM,The LOW-LAYER_COMMUNICATION_MODULE is a functi...,"709bb69a81ecc1f51a9d4420b7de6772,96fddbf4556ae...",2,4,bc91ac78fd434239b5c69ab16dac33f2,2,None,1,None,bc91ac78fd434239b5c69ab16dac33f2,0,0


In [4]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 239


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,IO_SAP,IO_CONTROL_PATH,8.0,IO SAP provides an interface for hardware I/O ...,[709bb69a81ecc1f51a9d4420b7de6772],e230f9d416db4a098a880d8e8f2419fe,0,1,1,2
1,INTER_SYSTEM_PROTOCOL,IEEE,7.0,The inter system protocol is referenced for mo...,[709bb69a81ecc1f51a9d4420b7de6772],2600fe0783574026a2dd216b10fce479,1,1,1,2
2,LOGICAL_NETWORK,EV,8.0,The EV must join the logical network of the EV...,[96fddbf4556ae84fd85ddd36d66bb9f0],58296d5c005a4fbcad34e69d22ed32dd,2,4,7,11
3,LOGICAL_NETWORK,EVSE,8.0,The EVSE must join the logical network of the ...,[96fddbf4556ae84fd85ddd36d66bb9f0],44f21ba2f1e24c66a21e8257fb08a112,3,4,5,9
4,LOGICAL_NETWORK,COMMUNICATION_NODE,8.0,The communication node manages the logical net...,[96fddbf4556ae84fd85ddd36d66bb9f0],51d9e2e0c6554a9dba3326304e1fe7a6,4,4,2,6


In [5]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 25


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,# CM_ATTEN_CHAR.RSP and TT_MATCH_RESPONSE_TIME...,1,9.0,CM_ATTEN_CHAR.RSP and TT_MATCH_RESPONSE_TIMER ...,The rating is high due to the critical role th...,The community revolves around the CM_ATTEN_CHA...,[{'explanation': 'The CM_ATTEN_CHAR.RSP messag...,"{\n ""title"": ""CM_ATTEN_CHAR.RSP and TT_MATC...",9261c28a-60ff-4730-886c-8be1f32eb54d
1,11,# CM_SLAC_PARM Communication and Test Cases\n\...,1,9.0,CM_SLAC_PARM Communication and Test Cases,The impact rating is high due to the critical ...,The community revolves around the CM_SLAC_PARM...,[{'explanation': 'CM_SLAC_PARM.CNF is a confir...,"{\n ""title"": ""CM_SLAC_PARM Communication an...",42c5cffb-7c8e-42cd-a2ac-c3112238b478
2,12,# ISO 15118-3:2015(E) and HomePlug Green PHY\n...,1,9.0,ISO 15118-3:2015(E) and HomePlug Green PHY,The rating is high due to the critical nature ...,The community revolves around the ISO 15118-3:...,[{'explanation': 'ISO 15118-3:2015(E) is the c...,"{\n ""title"": ""ISO 15118-3:2015(E) and HomeP...",8e120bba-dd68-422f-90ee-29b3070792ee
3,13,# ISO 15118-3:2015(E) and EVSE Communication N...,1,9.0,ISO 15118-3:2015(E) and EVSE Communication Net...,The rating is high due to the critical nature ...,The community revolves around the ISO 15118-3:...,[{'explanation': 'The SLAC Validation Process ...,"{\n ""title"": ""ISO 15118-3:2015(E) and EVSE ...",64b818cd-d2cd-420e-9985-2a7bb17c981a
4,14,# Electric Vehicle Communication Interfaces un...,1,9.0,Electric Vehicle Communication Interfaces unde...,The rating is high due to the critical importa...,The community revolves around the Electric Veh...,[{'explanation': 'The Electric Vehicle (EV) is...,"{\n ""title"": ""Electric Vehicle Communicatio...",a059abe2-9fd9-438b-b5c9-1c39f1747776


In [6]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 179


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,13530e8cc9d82af6d4cdfe790746f1a5,© ISO 2015\nRoad vehicles — Vehicle to grid \n...,1200,[40804f1931331b5055b914cf85f03b9f],None,None
1,f089a3eb01771db9f208ad6c266d776c,....28\n© ISO 2015 – All rights reserved\niii...,1200,[40804f1931331b5055b914cf85f03b9f],None,None
2,b82cb0d2eead2a86521f84c5e109d379,well as to support additional information ser...,1200,[40804f1931331b5055b914cf85f03b9f],None,None
3,709bb69a81ecc1f51a9d4420b7de6772,].\n3.14\nIO SAP\nIO control path interfaces h...,1200,[40804f1931331b5055b914cf85f03b9f],"[c1c991eb4a97411eb3d036a31cac6f2f, 5d50bcf3d46...","[e230f9d416db4a098a880d8e8f2419fe, 2600fe07835..."
4,df108cd528164964a36faf163c4b4240,15118-3:2015(E)\nMME\nManagement Message Entr...,1200,[40804f1931331b5055b914cf85f03b9f],None,None


In [7]:
import tiktoken
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType

# Directly assign values for API key, model, and embedding model
api_key = "375b0d5aca534b3c8b73cd797c3d56fe"
llm_model = "gpt-4o-2024-05-13"  # Example model name
embedding_model = "text-embedding-ada-002"  # Example embedding model

azure_llm_api_base = "https://test-text-embedding.openai.azure.com/openai/deployments/gpt-4o-2024-05-13/chat/completions?api-version=2023-03-15-preview"  # Azure API Base
azure_enc_api_base = "https://test-text-embedding.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"

azure_llm_deployment_name = "gpt-4o-2024-05-13"  # Deployment name for your model
azure_enc_deployment_name = "text-embedding-ada-002"  # Deployment name for your model


In [8]:
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
    api_base = azure_llm_api_base,
    api_version='2024-02-15-preview'
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=azure_enc_api_base,
    api_type=OpenaiApiType.AzureOpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
    api_version='2024-02-15-preview'
)

In [9]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    covariates=None,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [10]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [11]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [12]:
#generate light rag answer and write to a csv
import json
import pandas as pd
from tqdm import tqdm

with open('../QA.json', 'r') as f:
    QAset = json.load(f)

graphrag_ans_df = pd.DataFrame(columns=['question','generated_answer'])

question_list = []
generated_ans_list = []

for qa_pair in tqdm(QAset['qa_pairs']):
    cur_q = qa_pair['question']
    cur_ans = await search_engine.asearch(cur_q)

    question_list.append(cur_q)
    generated_ans_list.append(cur_ans.response)

graphrag_ans_df["question"] = question_list
graphrag_ans_df["generated_answer"] = generated_ans_list
graphrag_ans_df.to_csv('graphrag_ans.csv',index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [03:33<00:00,  7.13s/it]


In [16]:
result = await search_engine.asearch("What is D-LINK_READY.indication?")
print(result.response)

### Overview of D-LINK_READY.indication

The D-LINK_READY.indication is a message used within the context of data link layer communication, particularly in systems adhering to the ISO 15118 standard for electric vehicle (EV) communication. This message plays a crucial role in informing higher layers about the status of the communication link, which can either be 'link established' or 'no link' [Data: Entities (43); Sources (15)].

### Function and Purpose

The primary function of the D-LINK_READY.indication is to notify higher layers, such as the High-Level Entity (HLE), about changes in the status of the communication link. This notification is essential for the proper coordination and management of the communication process between the EV and the Electric Vehicle Supply Equipment (EVSE). When the communication link is established, the D-LINK_READY.indication informs the HLE that the binding process can begin. Conversely, when the link is terminated, it signals that the communication 